# Example script

Example script to show how to use the proposed MRF reconstruction.

In [ ]:
import h5py

from MRFrecon import MrfData

## Load data
First download the data from location as specified in ``example_data/readme.txt``.

### Load data from numerical phantom

In [ ]:
with h5py.File(r'example_data/num_phantom.h5', 'r') as hf:
    print(hf.keys())
    ksp = hf.get('kspace')[:]
    coord = hf.get('coord')[:]

# Load dictionary
with h5py.File(r'example_data/dictionary.h5', 'r') as hf:
    dictmat = hf.get('dictmat')[:].T.imag
    dictt1 = hf.get('t1list')[:]
    dictt2 = hf.get('t2list')[:]

### Create the data class

In [ ]:
data = MrfData()
data.ksp = ksp
data.coord = coord
data.dictmat = dictmat
data.dictt1 = dictt1
data.dictt2 = dictt2

In [ ]:
# equivalent to:
# data = MrfData(ksp, coord, dictmat, dictt1, dictt2)

In [ ]:
# make 2 copies
data2 = data.copy()

In [ ]:
data3 = data.copy()

# Solve LR inversion with single component matching

In [ ]:
# very similar to (slightly different settings)
#       recon_lr_invert(data, settings=load_settings('./example_data/config.ini',0))

In [ ]:
# compress dictionary to rank 10
data.compress_dictionary(10)

In [ ]:
# Set the compute device, 0 is a GPU
compute_device = 0

In [ ]:
# Espirit calc for sensitivity maps, with reconstruction matrix of size 256x256
data.mrf_espirit(256, compute_device=compute_device, tol_fac=.005, )

In [ ]:
# compress data to 5 virtual coils
data.coil_compression(5)

In [ ]:
# Low rank inversion image reconstruction
data.lr_inversion(compute_device=compute_device, tol_fac=.005, )

In [ ]:
# rotate image sequence to real axis
data.rotate2real()

In [ ]:
# Solve for single component MRF
data.single_component_match(verbose=2) # Verbose is 2 creates plots

In [ ]:
# Save to .h5
data.to_h5('./example_data/', 'lr_inv_single.h5')

# Solve LR ADMM with SPIJN component reconstruction

In [ ]:
# very similar to (slightly different settings)
# recon_admm_into_spijn(data, settings=load_settings('./example_data/config.ini',2))

In [ ]:
# compress dictionary to rank 10
data2.compress_dictionary(10)

In [ ]:
# Espirit calc for sensitivity maps, with reconstruction matrix of size 256
data2.mrf_espirit(256, compute_device=compute_device)

In [ ]:
# compress data to 5 virtual coils
data2.coil_compression(5)

In [ ]:
# Low rank admm image reconstruction
data2.lr_admm(2e-3, compute_device=compute_device, n_jobs=1, max_iter=5, max_cg_iter=50,
              tol_fac=0.001, lstsq_solver = 'PrimalDualHybridGradient')
# Note: low rank admm does not need rotate2real()!

In [ ]:
# Solve for components with a joint sparse regularization parameter of 0.3
data2.spijn_solve(0.3, verbose=2, n_jobs=4)

In [ ]:
# Save to .h5
data2.to_h5('./example_data/', 'lr_admm_spijn.h5')

# k-SPIJN direct MC-MRF reconstruction

In [ ]:
# very similar to (slightly different settings)
# recon_admm_into_spijn(data, settings=load_settings('./example_data/config.ini',1))

In [ ]:
# compress dictionary to rank 10
data3.compress_dictionary(10)

In [ ]:
# Espirit calc for sensitivity maps, with reconstruction matrix of size 256
data3.mrf_espirit(256, compute_device=compute_device)

In [ ]:
# compress data to 5 virtual coils
data3.coil_compression(5)

In [ ]:
# Direct component reconstruction
data3.spijn_from_ksp(admm_param=2e-3, reg_param=0.1,
                     compute_device=compute_device, verbose=2, n_jobs=1, 
                     tol_fac=.001, tol=1e-3)

In [ ]:
data3.plot_comp()